# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [2]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [3]:
df = pd.read_excel(r'C:\Users\user\предпосылки\Экс10.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.000000,0.200000,0.333333,7,1.000000,0.333333,0.200000
1,Экспертиза промышленной безопасности,5.000000,1.000000,9.000000,9,5.000000,5.000000,5.000000
2,Готовность к ликвидации последствий аварий и и...,3.000000,0.111111,1.000000,4,0.200000,3.000000,0.200000
3,Оформление информационными знаками и знаками б...,0.142857,0.111111,0.250000,1,0.111111,0.200000,0.111111
4,Содержание объектов и оборудования,1.000000,0.200000,5.000000,9,1.000000,7.000000,0.333333


In [4]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [5]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности      int64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала       float64
dtype: object

In [6]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0       1.000000                              0.200000   
1       5.000000                              1.000000   
2       3.000000                              0.111111   
3       0.142857                              0.111111   
4       1.000000                              0.200000   
5       3.000000                              0.200000   
6       5.000000                              0.200000   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                           0.333333         
1                                           9.000000         
2                                           1.000000         
3                                           0.250000         
4                                           5.000000         
5                                           0.333333         
6                                           5.000000         

   Оформление информационными знаками 

In [7]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности      int64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала       float64
dtype: object

In [8]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [9]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,1.000000,0.200000,0.333333,1.000000,0.333333,0.200000
1,5.000000,1.000000,9.000000,5.000000,5.000000,5.000000
2,3.000000,0.111111,1.000000,0.200000,3.000000,0.200000
3,0.142857,0.111111,0.250000,0.111111,0.200000,0.111111
4,1.000000,0.200000,5.000000,1.000000,7.000000,0.333333
5,3.000000,0.200000,0.333333,0.142857,1.000000,0.111111
6,5.000000,0.200000,5.000000,3.000000,9.000000,1.000000


In [10]:
DF_1=df[:].copy()

In [11]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                 18.142857
Экспертиза промышленной безопасности                          2.022222
Готовность к ликвидации последствий аварий и инцидентов      20.916667
Оформление информационными знаками и знаками безопасности    44.000000
Содержание объектов и оборудования                           10.453968
Охранные зоны (ОЗ)                                           25.533333
Организация рабочего процесса и безопасности персонала        6.955556
dtype: float64

In [12]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [13]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.000000,0.200000,0.333333,7,1.000000,0.333333,0.200000,0.055118,0.098901,0.015936,0.159091,0.095657,0.013055,0.028754
1,5.000000,1.000000,9.000000,9,5.000000,5.000000,5.000000,0.275591,0.494505,0.430279,0.204545,0.478287,0.195822,0.718850
2,3.000000,0.111111,1.000000,4,0.200000,3.000000,0.200000,0.165354,0.054945,0.047809,0.090909,0.019131,0.117493,0.028754
3,0.142857,0.111111,0.250000,1,0.111111,0.200000,0.111111,0.007874,0.054945,0.011952,0.022727,0.010629,0.007833,0.015974
4,1.000000,0.200000,5.000000,9,1.000000,7.000000,0.333333,0.055118,0.098901,0.239044,0.204545,0.095657,0.274151,0.047923


In [14]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.055118,0.098901,0.015936,0.159091,0.095657,0.013055,0.028754
1,0.275591,0.494505,0.430279,0.204545,0.478287,0.195822,0.718850
2,0.165354,0.054945,0.047809,0.090909,0.019131,0.117493,0.028754
3,0.007874,0.054945,0.011952,0.022727,0.010629,0.007833,0.015974
4,0.055118,0.098901,0.239044,0.204545,0.095657,0.274151,0.047923
5,0.165354,0.098901,0.015936,0.113636,0.013665,0.039164,0.015974
6,0.275591,0.098901,0.239044,0.204545,0.286972,0.352480,0.143770


In [15]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.055118,0.098901,0.015936,0.159091,0.095657,0.013055,0.028754,0.066645
1,0.275591,0.494505,0.430279,0.204545,0.478287,0.195822,0.718850,0.399697
2,0.165354,0.054945,0.047809,0.090909,0.019131,0.117493,0.028754,0.074914
3,0.007874,0.054945,0.011952,0.022727,0.010629,0.007833,0.015974,0.018848
4,0.055118,0.098901,0.239044,0.204545,0.095657,0.274151,0.047923,0.145049


In [16]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [17]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [18]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

0.5163201356906458
3.776257556036002
0.6676819276881593
0.14625978215368657
1.3747164093369295
0.5113126733279014
2.216078189755099


In [19]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

9.208626673988423


In [20]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

0.36810444566473716
1.4142857142857144
0.26027587067203634


In [21]:
# Получилось, что значение CR=0.26, что большее 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами